# Training of the Extended Mask2Former

In [5]:
# Import Libraries
import sys, os
module_path = os.path.abspath(os.path.join('..'))
from torchvision.datasets import CocoDetection
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from PIL import Image
from models.extended_mask2former_model import ExtendedMask2Former
from src.utils import metrics

# Training parameters
EPOCHS = 10
BATCH_SIZE = 16
LEARNING_RATE = 1e-3
BASE_DATA_PATH = "data/coco2017"

### Set up GPU growth

In [2]:
# Set device we are going to load the model and the data
device = 'mps' if torch.backends.mps.is_available() else "cpu"

### Data Preprocessing 

In [9]:
# Get the mean and standard deviation from the COCO dataset
def custom_collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch)) 
    images, targets = zip(*batch)
    images = torch.stack(images, dim=0)        
    return images, targets



# Define your transformations
transform = transforms.Compose([
    transforms.Resize((600, 600)),
    transforms.ToTensor()
])

coco_train = CocoDetection(root='data/coco2017/images/train2017', annFile='data/coco2017/annotations/instances_train2017.json', transform=transform)
coco_val = CocoDetection(root='data/coco2017/images/val2017',  annFile='data/coco2017/annotations/instances_val2017.json', transform=transform)

val_loader = DataLoader(dataset=coco_val, batch_size=4, num_workers=1)

mean, standard_deviation = metrics.calculate_data_mean_std(val_loader)
print(mean)
print(standard_deviation)







loading annotations into memory...
Done (t=6.57s)
creating index...
index created!
loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/stamatiosorphanos/miniconda3/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/Users/stamatiosorphanos/miniconda3/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/Users/stamatiosorphanos/miniconda3/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/Users/stamatiosorphanos/miniconda3/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 142, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "/Users/stamatiosorphanos/miniconda3/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line

In [ ]:
# Import necessary libraries
# Initialize the COCO dataset
# root_dir = 'data/coco2017/images/train2017'
# annotation_file = 'data/coco2017/annotations/instances_train2017.json'  # Update this path

# coco_dataset = COCODataset(root_dir=root_dir, annotation_file=annotation_file, transform=transform)
# data_loader = DataLoader(coco_dataset, batch_size=4, shuffle=True, num_workers=4)

# # Define your model, loss function, and optimizer Initialize your model
# num_classes = 80
# model = ExtendedMask2Former(num_classes=num_classes)  
# model.to(device)
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# loss_fn = metrics.calculate_loss



# # Training loop (simplified)
# num_epochs = 10
# for epoch in range(num_epochs):
#     for images, targets in data_loader:
#         images = images.to(device)
#         targets = targets.to(device)
        
#         # Forward pass
#         outputs = model(images)
        
#         # Compute loss
#         loss = loss_fn(outputs, targets)
        
#         # Backward pass and optimize
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
    
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# # Note: This is a very simplified training loop. You'll need to adjust it based on your model's specific inputs and outputs.
